In [14]:
#%matplotlib inline
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import time
import random
import copy
from copy import deepcopy
import threading

from train_utils import *

import matplotlib.pyplot as plt
import cv2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from torchvision import transforms
from IPython.core.display import Image as JupyterImage
from torchvision.utils import save_image

deg_to_rad = lambda x: x*0.0174533

crop = transforms.CenterCrop(56)
resize = transforms.Resize(64)
color_jitter = transforms.ColorJitter(brightness=.5, contrast=.5, saturation=.5, hue=.5)

def get_rotated_view(front, aux, rotation):
    front = front.clone()
    SEQ_LEN, BS, C, H, W = front.shape
    ff = front.reshape(SEQ_LEN*BS, C, H, W)
    ff = transforms.RandomRotation([rotation,rotation], expand=False, center=None, fill=None)(ff)
    ff = crop(ff)
    ff = resize(ff)
    #ff = color_jitter(ff)
    ff = ff.reshape(SEQ_LEN, BS, C, H, W)
    _aux = aux.clone()
    _aux[:,:,0] -= deg_to_rad(rotation)
    return ff, _aux

In [16]:
import gym3
from procgen import ProcgenGym3Env

In [17]:
bs = 1

# num_levels=1, start_level=6
env = ProcgenGym3Env(num=bs, env_name="testgame", render_mode='rgb_array')
env = gym3.ViewerWrapper(env, info_key="rgb")

In [22]:
device = 'cuda'
m = VizCNN(use_rnn=False).to(device);
m.load_state_dict(torch.load("m_cons_3.28b.torch"))
m.eval()

RuntimeError: [enforce fail at inline_container.cc:222] . file not found: archive/data/93987830202736

In [12]:
hidden = get_hidden(1)

In [13]:
%%time

use_autopilot = True

s = np.array([[.1, .2] for _ in range(bs)], dtype=np.float32)
daggerized_controls = s


seq_len = 1000
DAGGER_CADENCE = 100
DAGGER_DURATION = 10
do_dagger = False
dagger_counter = 0
TRAINING_WHEELS_WINDOW = 20
use_training_wheels = False

act_grads = []
imgs = []
salmaps = []

#with torch.no_grad():
for i in range(seq_len):
    env.act(s)
    rew, obs, first = env.observe()
    img = obs['rgb']
    info = env.get_info()

    aux = np.array([[e[a] for a in aux_properties] for e in info])
    autopilot_controls = np.array([[e['autopilot_steer'], e['autopilot_throttle']] for e in info])

    front = torch.from_numpy(img.astype(np.float32)/255.).unsqueeze(0).permute(0,1,4,2,3)
    aux = torch.from_numpy(aux.astype(np.float32)).unsqueeze(0)

    if use_autopilot:
        if i % DAGGER_CADENCE == 0:
            print('SETTING DAGGER')
            dagger_counter = 0
            steer_aug = random.uniform(-.3, .3)
            #throttle_aug = random.uniform(.5, 1.5)
            daggerized_controls = np.array([[c[0]+steer_aug, c[1]] for c in autopilot_controls])
            do_dagger = True

        if do_dagger:
            s = daggerized_controls
            dagger_counter+=1
        else:
            s = autopilot_controls

        if dagger_counter == DAGGER_DURATION:
            do_dagger = False

    else:
        front = front.to(device)
        aux = aux.to(device)
        out,hidden, salmap = m(front, aux, hidden, return_salmap=True, register_activations=True)
        out[:,:,1]=.6
        s = out.squeeze(0).cpu().detach().numpy()
        
        
        """        baseline = torch.zeros_like(front)
        steps = 100
        grads = []

        for i in range(0,steps+1):
            mixed_img = baseline + (float(i)/steps)*(front-baseline)
            m.zero_grad()
            out, hidden, salmap = m(mixed_img, aux, hidden, return_salmap=True, register_activations=True)
            out[0][0][0].backward(retain_graph=m.use_rnn) # Steer
            g = m.get_activations_gradient()
            g = g[0].mean(0).cpu().numpy()
            grads.append(g)

        avg_grads = np.stack(grads).mean(0)
        act_grad = avg_grads
        
        salmap = salmap[0].mean(0)
        
        act_grads.append(act_grad * salmap)
        #act_grads.append(salmap) # This by itself is pretty nice
        
        imgs.append(img[0])"""

    if use_training_wheels and i < TRAINING_WHEELS_WINDOW:
        s = autopilot_controls
        

SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
SETTING DAGGER
CPU times: user 2.75 s, sys: 174 ms, total: 2.92 s
Wall time: 1min 6s


In [7]:

def img_cam(act_grad, img, std):
    # Gradients zero to one
    act_grad = cv2.resize(act_grad, (img.shape[0],img.shape[1]))
    mask = np.where(act_grad, (abs(act_grad)>std*2), 0)
    mask = np.expand_dims(mask, -1)
    
    act_grad -= act_grad.min()
    act_grad = act_grad / act_grad.max()
    
    # Make a three-channel heatmap out of the one channel gradients
    heatmap = cv2.applyColorMap(np.uint8(255 * act_grad), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    heatmap = heatmap * mask
    heatmap = heatmap * 255
    
    #img = img / 255
    
    cam = heatmap*.5 + img
    
    #cam = cam / cam.max()
    #cam = cam * 255
    cam = np.clip(cam, 0, 255)
    cam = cam.astype(np.uint8)
    
    return cam

In [8]:
std = np.array(act_grads[20:]).std()
img_cams = []
for img, act_grad in zip(imgs,act_grads):
    img_cams.append(img_cam(act_grad,img,std))

/home/beans/anaconda3/envs/procgen/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/beans/anaconda3/envs/procgen/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/beans/anaconda3/envs/procgen/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
img1 = img_cams[0]
height , width , layers = img1.shape
fps = 10

video = cv2.VideoWriter('cams.avi', cv2.VideoWriter_fourcc(*"MJPG"), fps, (width,height))

for i in range(len(img_cams)-1):
    img = img_cams[i] # cv2 expects out of 255, integers
    video.write(img)
    
cv2.destroyAllWindows()
video.release()

In [10]:
salmap.shape

(1, 16, 29, 29)